In [1]:
import MetaTrader5 as mt5
import pandas as pd 
mt5.initialize()

True

In [2]:
formato_numerico = "{0:10.2f}"
ativo = list(())
AtivoSymbolInfo = list(())
matriz = list(())
global matriz

In [3]:
def imprimirInfoAtivo(t, y, n):
    print(t, y, formato_numerico.format(n))

In [4]:
# ****************************************************
def RolarOpcoes(ativo):
    n = len(ativo)
    AtivoSymbolInfo.clear()
    rows,cols = (n,9)
    matriz = [[0 for i in range(cols)] for j in range(rows)]
    
    for i in range(n):
        AtivoSymbolInfo.append(mt5.symbol_info(ativo[i]))
        xx = AtivoSymbolInfo[i]
        matriz[i][0] = xx.name
        matriz[i][1] = xx.option_strike
        matriz[i][4] = xx.last
 
        # somente para linha 1, que é da opção vendida, que preciso rolar
        # pega o ASK... mas se não tiver ninguém oferecendo, pega o LAST
        if i == 1:
            matriz[1][3] = xx.ask
            if matriz[1][3] == 0:
                fff = input('Entre o valor do Ask ' + xx.name + ': ')
                matriz[i][3] = float(fff)
            gasto = matriz[1][3] * -1
#            if matriz[1][3] == 0:       gasto = matriz[1][4]
 
        # para as linhas 2,3,... ou seja, para as opções dos meses seguintes que poderei vender
        if i>1:
            matriz[i][2] = xx.bid
            if matriz[i][2] == 0:
                fff = input('Entre o valor do Bid ' + xx.name + ': ')
                matriz[i][2] = float(fff)
            matriz[i][5] = matriz[i][1] - matriz[1][1] # aumento em R$ strike nova - strike vendida
            matriz[i][6] = (matriz[i][1]/matriz[1][1] - 1) * 100  # % strike nova - strike vendido
            matriz[i][7] = matriz[i][5] + gasto + matriz[i][2] # dif.strike - gasto
            matriz[i][8] = matriz[i][7] / matriz[1][1] * 100

    '''for l in range(n):
        for c in range(9):
            print(f'|{matriz[l][c]:^10}|',end='')
        print()
    '''
    df = pd.DataFrame(matriz, columns=['Ativo','Strike','bid','ask','Last','▲ R$','▲ % Strike','Somatório','% sobre '+matriz[1][0]])
    print(df)
    nome_arq = 'rolar_opcoes_' + ativo[0] + '_' + ativo[1] + '_' + ativo[2]+ '.xlsx'
    df.to_excel(nome_arq)

In [5]:
ativo = ['CMIG4', 'CMIGG154', 'CMIGH139'] 
RolarOpcoes(ativo)

      Ativo  Strike   bid   ask   Last  ▲ R$  ▲ % Strike  Somatório  \
0     CMIG4    0.00  0.00  0.00  12.76  0.00    0.000000       0.00   
1  CMIGG154   13.35  0.00  0.26   0.28  0.00    0.000000       0.00   
2  CMIGH139   14.00  0.09  0.00   0.32  0.65    4.868914       0.48   

   % sobre CMIGG154  
0          0.000000  
1          0.000000  
2          3.595506  


In [6]:
ativo = ['CMIG4', 'CMIGG154', 'CMIGH134'] 
RolarOpcoes(ativo)

      Ativo  Strike   bid   ask   Last  ▲ R$  ▲ % Strike  Somatório  \
0     CMIG4    0.00  0.00  0.00  12.76  0.00    0.000000       0.00   
1  CMIGG154   13.35  0.00  0.26   0.28  0.00    0.000000       0.00   
2  CMIGH134   13.50  0.45  0.00   0.50  0.15    1.123596       0.34   

   % sobre CMIGG154  
0          0.000000  
1          0.000000  
2          2.546816  


In [7]:
ativo = ['CMIG4', 'CMIGG154', 'CMIGH131'] 
RolarOpcoes(ativo)

      Ativo  Strike   bid   ask   Last  ▲ R$  ▲ % Strike  Somatório  \
0     CMIG4    0.00  0.00  0.00  12.76   0.0    0.000000       0.00   
1  CMIGG154   13.35  0.00  0.26   0.28   0.0    0.000000       0.00   
2  CMIGH131   13.25  0.54  0.00   0.57  -0.1   -0.749064       0.18   

   % sobre CMIGG154  
0          0.000000  
1          0.000000  
2          1.348315  


In [8]:
ativo = ['ITSA4', 'ITSAG113', 'ITSAH115'] 
RolarOpcoes(ativo)

      Ativo  Strike  bid   ask   Last  ▲ R$  ▲ % Strike  Somatório  \
0     ITSA4    0.00  0.0  0.00  11.82  0.00    0.000000       0.00   
1  ITSAG113   11.26  0.0  0.79   0.65  0.00    0.000000       0.00   
2  ITSAH115   11.54  0.7  0.00   0.65  0.28    2.486679       0.19   

   % sobre ITSAG113  
0          0.000000  
1          0.000000  
2          1.687389  


In [9]:
ativo = ['CSAN3', 'CSANG100', 'CSANH250'] 
RolarOpcoes(ativo)

      Ativo  Strike   bid  ask   Last  ▲ R$  ▲ % Strike  Somatório  \
0     CSAN3    0.00  0.00  0.0  24.99  0.00     0.00000       0.00   
1  CSANG100   24.74  0.00  2.0   1.28  0.00     0.00000       0.00   
2  CSANH250   25.00  1.36  0.0   1.76  0.26     1.05093      -0.38   

   % sobre CSANG100  
0          0.000000  
1          0.000000  
2         -1.535974  


In [10]:
ativo = ['BBSE3', 'BBSET220', 'BBSET230'] 
RolarOpcoes(ativo)

      Ativo  Strike   bid   ask   Last  ▲ R$  ▲ % Strike  Somatório  \
0     BBSE3     0.0  0.00  0.00  25.41   0.0    0.000000       0.00   
1  BBSET220    22.0  0.00  0.25   0.00   0.0    0.000000       0.00   
2  BBSET230    23.0  0.16  0.00   0.00   1.0    4.545455       0.91   

   % sobre BBSET220  
0          0.000000  
1          0.000000  
2          4.136364  


In [11]:
at = mt5.symbols_get()
type(at)

tuple

In [12]:
print(len(at))
for i in range(5):   #    (len(at)):
    print(at[i].basis, at[i].name)

75428
ABEV3 ABEVA207
ABEV3 ABEVA212
ABEV3 ABEVA216
ABEV3 ABEVA219
ABEV3 ABEVA221
